### Bibliotecas e Funções

In [1]:
import numpy as np
import pandas as pd
import pulp

### Leitura dos dados

In [2]:
# Carrega os dados dos arquivos Excel
# parameters: parâmetros gerais do problema, como capacidade de estoque e tempo máximo de envelhecimento
parameters = pd.read_excel(io="Dataset_Desafio_v2.xlsx", sheet_name="parameters", index_col='Name')
# items_data: informações sobre os itens, como quantidade mínima e máxima de pedido
items_data = pd.read_excel(io="Dataset_Desafio_v2.xlsx", sheet_name="items")
# procurement_costs_data: custos de aquisição dos itens por período
procurement_costs_data = pd.read_excel(io="Dataset_Desafio_v2.xlsx", sheet_name="procurement_costs")
# demand_data: demanda dos itens por período
demand_data = pd.read_excel(io="Dataset_Desafio_v2.xlsx", sheet_name="demand")
# inventory_data: informações sobre estoque, como custos de armazenamento e estoque inicial
inventory_data = pd.read_excel(io="Dataset_Desafio_v2.xlsx", sheet_name="inventory")

# Cria o problema de minimização de custos usando a biblioteca pulp
prob = pulp.LpProblem(name="Minimização de custos", sense=pulp.LpMinimize)

# Define o número de produtos (N_PRODUCTS)
# Define o número de períodos (N_PERIODS)
# Define o tempo máximo de envelhecimento (MAX_AGING_TIME) - período máximo que um item pode ficar no estoque do fornecedor
# Define a capacidade de recebimento do depósito da fábrica (WAREHOUSE_RECEIVING_CAPACITY) - número máximo de tipos de itens que podem ser recebidos por período
# Define uma constante BIG_M, um valor grande usado em algumas restrições para modelar decisões lógicas
BIG_M = 1e4

N_PRODUCTS = 5 # Fixando o número de produtos para 5 para testes
#N_PERIODS = 20 # Fixando o número de períodos para 20 para testes
#MAX_AGING_TIME = 5 # Fixando o tempo máximo de envelhecimento para 5 para testes
#WAREHOUSE_RECEIVING_CAPACITY = 3 # Fixando a capacidade de recebimento para 3 para testes

# Extrai os valores dos parâmetros relevantes do DataFrame parameters
# Número de produtos distintos (N_PRODUCTS) - extraído dos dados de itens
N_PRODUCTS = items_data.shape[0]
# Número de períodos de tempo no horizonte de planejamento (N_PERIODS) - extraído dos dados de custos de aquisição
N_PERIODS = len(procurement_costs_data['Period ID'].unique())
# Tempo máximo de envelhecimento (MAX_AGING_TIME) - parâmetro geral do problema
MAX_AGING_TIME = parameters.loc['Max Aging Time'].iloc[0]
# Capacidade de expedição do fornecedor por período (SUPPLIER_EXPEDITION_CAPACITY) - parâmetro geral do problema
SUPPLIER_EXPEDITION_CAPACITY = parameters.loc['Supplier Expedition Capacity'].iloc[0]
# Capacidade de recebimento do depósito da fábrica (WAREHOUSE_RECEIVING_CAPACITY) - parâmetro geral do problema
WAREHOUSE_RECEIVING_CAPACITY = parameters.loc['Warehouse Receiving Capacity'].iloc[0]
# Capacidade de estoque do fornecedor (SUPPLIER_INVENTORY_CAPACITY) - parâmetro geral do problema
SUPPLIER_INVENTORY_CAPACITY = parameters.loc['Supplier Inventory Capacity'].iloc[0]
# Capacidade de estoque da fábrica (WAREHOUSE_INVENTORY_CAPACITY) - parâmetro geral do problema
WAREHOUSE_INVENTORY_CAPACITY = parameters.loc['Warehouse Inventory Capacity'].iloc[0]

# Cria dicionários para armazenar informações extraídas de DataFrames

# Dicionário para armazenar a quantidade mínima de pedido (MinO) para cada item
MinO = {row["Item ID"]: row["Min Order Qty."] for _, row in items_data.iterrows()}
# Dicionário para armazenar a quantidade máxima de pedido (MaxO) para cada item
MaxO = {row["Item ID"]: row["Max Order Qty."] for _, row in items_data.iterrows()}
# Dicionário para armazenar a quantidade mínima de transferência (MinT) para cada item
MinT = {row["Item ID"]: row["Min Transfer Qty."] for _, row in items_data.iterrows()}

# Dicionário para armazenar o custo unitário de aquisição (P) para cada item e período
P = {(row["Item ID"], row["Period ID"]): row["Unit Cost"] for _, row in procurement_costs_data.iterrows()}
# Dicionário para armazenar a demanda (D) para cada item e período
D = {(row["Item ID"], row["Period ID"]): row["Demand Qty."] for _, row in demand_data.iterrows()}
# Dicionário para armazenar o estoque mínimo (M) para cada item e período
M = {(row["Item ID"], row["Period ID"]): row["Min Inventory"] for _, row in demand_data.iterrows()}

# Dicionário para armazenar o custo unitário de armazenamento no fornecedor (CS) para cada item
CS = {row["Item ID"]: row["Unit Holding Cost"] for _, row in inventory_data.iterrows() if row["Site ID"] == "S"}
# Dicionário para armazenar o custo unitário de armazenamento na fábrica (CW) para cada item
CW = {row["Item ID"]: row["Unit Holding Cost"] for _, row in inventory_data.iterrows() if row["Site ID"] == "WH"}

# Define variáveis de decisão usando a biblioteca pulp
# X: quantidade de cada item comprada em cada período
# W: quantidade de cada item no estoque da fábrica em cada período
# S: quantidade de cada item no estoque do fornecedor em cada período
# T: quantidade de cada item transportada do fornecedor para a fábrica em cada período
# Y: variável binária que indica se houve transferência de um item do fornecedor para a fábrica em um período
# Z: variável binária que indica se houve compra de um item em um período

X = pulp.LpVariable.dicts(name="X",
    indices=((f'B{i}', t) for i in range(1, N_PRODUCTS+1) for t in range(0, N_PERIODS+1)),
    lowBound=0,
    cat='Integer'
)

W = pulp.LpVariable.dicts(name="W",
    indices=((f'B{i}', t) for i in range(1, N_PRODUCTS+1) for t in range(0, N_PERIODS+1)),
    lowBound=0,
    cat='Integer'
)

S = pulp.LpVariable.dicts(name="S",
    indices=((f'B{i}', t) for i in range(1, N_PRODUCTS+1) for t in range(0, N_PERIODS+1)),
    lowBound=0,
    cat='Integer'
)

T = pulp.LpVariable.dicts(name="T",
    indices=((f'B{i}', t) for i in range(1, N_PRODUCTS+1) for t in range(0, N_PERIODS+1)),
    lowBound=0,
    cat='Integer'
)

Y = pulp.LpVariable.dicts(name="Y",
    indices=((f'B{i}', t) for i in range(1, N_PRODUCTS+1) for t in range(0, N_PERIODS+1)),
    lowBound=0,
    upBound=1,
    cat='Integer'
)

Z = pulp.LpVariable.dicts(name="Z",
    indices=((f'B{i}', t)for i in range(1, N_PRODUCTS+1) for t in range(0, N_PERIODS+1)),
    lowBound=0,
    upBound=1,
    cat='Integer'
)

# Inicializa as variáveis W e S no período 0 com os dados de estoque inicial
# W: inicializa a quantidade de cada item no estoque da fábrica no período 0 (W[(f'B{i}', 0)])
# com base nos dados de estoque inicial do arquivo inventory_data.
# O valor é fixado para garantir que a otimização comece com os estoques iniciais corretos.
# S: inicializa a quantidade de cada item no estoque do fornecedor no período 0 (S[(f'B{i}', 0)])
# com base nos dados de estoque inicial do arquivo inventory_data.
# O valor é fixado para garantir que a otimização comece com os estoques iniciais corretos.

initial_inventory_wh = {row["Item ID"]: row["Opening Inventory"] for _, row in inventory_data.iterrows() if row["Site ID"] == "WH"}
for i in range(1, N_PRODUCTS+1):
    W[(f'B{i}', 0)].setInitialValue(initial_inventory_wh.get(f'B{i}', 0))
    W[(f'B{i}', 0)].fixValue()

initial_inventory_s = { row["Item ID"]: row["Opening Inventory"] for _, row in inventory_data.iterrows() if row["Site ID"] == "S"}
for i in range(1, N_PRODUCTS+1):
    S[(f'B{i}', 0)].setInitialValue(initial_inventory_s.get(f'B{i}', 0))
    S[(f'B{i}', 0)].fixValue()

# Define a função objetivo - minimizar o custo total, que é a soma dos custos de aquisição, armazenamento no fornecedor e armazenamento na fábrica
# O custo total é calculado para cada item em cada período.

prob += pulp.lpSum(
    [P[f'B{i}', t] * X[(f'B{i}', t)] + CS[f'B{i}'] * S[(f'B{i}', t)] + CW[f'B{i}'] * W[(f'B{i}', t)]
    for t in range(1, N_PERIODS+1)
    for i in range(1, N_PRODUCTS+1)
])

# Define as restrições do modelo para garantir que a solução seja viável e respeite as regras operacionais do problema

for t in range(1, N_PERIODS+1):
    #Restrições de Aquisição: limita o número máximo de diferentes tipos de itens (produtos) que podem ser adquiridos em um período (t).
    prob += pulp.lpSum(Z[(f'B{i}', t)] for i in range(1, N_PRODUCTS+1)) <= WAREHOUSE_RECEIVING_CAPACITY

    #Restrições de Capacidade de Estoque do Fornecedor: assegura que o estoque do fornecedor não exceda sua capacidade em cada período (t).
    prob += pulp.lpSum(S[(f'B{i}', t)] for i in range(1, N_PRODUCTS+1)) <= SUPPLIER_INVENTORY_CAPACITY

    #Restrições de Capacidade de Estoque da Fábrica: garante que o estoque da fábrica não ultrapasse sua capacidade em cada período (t).
    prob += pulp.lpSum(W[(f'B{i}', t)] for i in range(1, N_PRODUCTS+1)) <= WAREHOUSE_INVENTORY_CAPACITY

    #Restrições de Capacidade de Transporte do Fornecedor: limita a quantidade total de itens que o fornecedor pode enviar à fábrica em cada período (t).
    prob += pulp.lpSum(T[(f'B{i}', t)] for i in range(1, N_PRODUCTS+1)) <= SUPPLIER_EXPEDITION_CAPACITY

    #Restrições de Capacidade de Recebimento da Fábrica: limita a quantidade de diferentes tipos de itens que a fábrica pode receber do fornecedor em cada período (t).
    prob += pulp.lpSum(Y[(f'B{i}', t)] for i in range(1, N_PRODUCTS+1)) <= WAREHOUSE_RECEIVING_CAPACITY


for i in range(1, N_PRODUCTS+1):
    #Restrições de Quantidade de Aquisição - Limite Superior: certifica que a quantidade comprada de um item não exceda a quantidade máxima de pedido.
    #Utiliza a variável Z para indicar se houve compra e limita a compra à quantidade máxima (MaxO) quando Z é 1 (houve compra).
    prob += X[(f'B{i}', t)] <= MaxO[f'B{i}'] * Z[(f'B{i}', t)]
    #Restrições de Quantidade de Aquisição - Limite Inferior (relacionada à variável Z): garante que, se houver compra de um item (Z=1), a quantidade mínima de pedido seja respeitada.
    prob += X[(f'B{i}', t)] >= 1 * Z[(f'B{i}', t)]
    #Restrições de Quantidade de Aquisição - Limite Inferior: certifica que, se não houver compra de um item (Z=0), a quantidade adquirida seja zero.
    prob += X[(f'B{i}', t)] >= MinO[f'B{i}'] - BIG_M * (1-Z[(f'B{i}', t)])


    #Restrições de Balanço de Estoque do Fornecedor: garante que o estoque do fornecedor em um período seja igual ao estoque do período anterior,
    #acrescido da quantidade comprada e subtraído da quantidade transferida para a fábrica.
    prob += S[(f'B{i}', t-1)] + X[(f'B{i}', t)] == T[(f'B{i}', t)] + S[(f'B{i}', t)]


    #Restrições de Balanço de Estoque da Fábrica: assegura que o estoque da fábrica em um período seja igual ao estoque do período anterior,
    #acrescido da quantidade recebida do fornecedor e subtraído da demanda.
    prob += W[(f'B{i}', t-1)] + T[(f'B{i}', t)] == D.get((f'B{i}', t), 0) + W[(f'B{i}', t)]
    #Restrições de Estoque Mínimo da Fábrica: garante que o estoque da fábrica de um item em cada período seja igual ou superior ao estoque mínimo definido para o item.
    prob += W[(f'B{i}', t)] >= M.get((f'B{i}', t), 0)

    #Restrições de Quantidade de Transferência - Limite Inferior (relacionada à variável Y): garante que a transferência do fornecedor para a fábrica ocorra apenas se Y[(f'B{i}', t)] for igual a 1.
    #A quantidade transferida será maior ou igual à quantidade mínima de transferência, ou zero caso contrário (Y[(f'B{i}', t)]=0).
    prob += T[(f'B{i}', t)] >= MinT[f'B{i}'] - BIG_M * (1-Y[(f'B{i}', t)])
    #Restrições de Quantidade de Transferência - Limite Superior: limita a quantidade transferida de um item, em cada período, à capacidade de expedição do fornecedor.
    prob += T[(f'B{i}', t)] <= \
    SUPPLIER_EXPEDITION_CAPACITY * Y[(f'B{i}', t)]
    #Restrições de Quantidade de Transferência - Limite Inferior: garante que a transferência, se ocorrer (Y=1), seja de pelo menos uma unidade do item.
    prob += T[(f'B{i}', t)] >= 1 * Y[(f'B{i}', t)]


c:\Projetos\MipWise-Dogs\.venv\Lib\site-packages\pulp\pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [3]:
solverList = pulp.listSolvers(onlyAvailable=True)
solverList

['PULP_CBC_CMD']

In [4]:
# Resolvendo o problema de otimização utilizando o solver CBC
prob.solve(solver=pulp.getSolver('PULP_CBC_CMD'))

# Imprimindo o status da solução do problema
print("Status:", pulp.LpStatus[prob.status])

# Verificando se a solução é viável
if pulp.LpStatus[prob.status] == 'Infeasible':
    print('\nSem Solução')
else:
    # Imprimindo o valor ótimo da função objetivo
    print("Custo ótimo:", pulp.value(prob.objective))


Status: Optimal
Custo ótimo: 32420.019999999997


In [5]:
# Preparando para extrair os resultados da solução
NewMinO = {}
for t in range(0, N_PERIODS + 1):
    for i in range(1, N_PRODUCTS + 1):
        # Armazenando o estoque mínimo por produto e período
        NewMinO[(f'B{i}', t)] = MinO[f'B{i}']

# Criando listas para armazenar os valores das variáveis de decisão
data = [X, Z, S, T, Y, W]

# Criando um DataFrame a partir dos dados das variáveis de decisão
df = pd.DataFrame.from_records(data, index=['Aquisição', 'Comprar', 'Armazém Fornecedor', 'Transporte', 'Transportar', 'Armazém Fábrica'])
df = df.T # Transpondo o DataFrame



In [8]:
# Função para extrair o valor da variável
def getvalue(x):
    return x.varValue

# Aplicando a função getvalue para obter os valores das variáveis
for key in df.columns:
    df[key] = df[key].apply(getvalue)

# Configurando o índice do DataFrame com Produto e Período
df.index = pd.MultiIndex.from_tuples(df.index, names=['Produto', 'Período'])
df = df.fillna(0) # Preenchendo valores ausentes com 0

# Configurando a exibição do DataFrame
pd.options.display.max_rows = 999
# Exibindo apenas os períodos de 0 a 9
print(df[df.index.get_level_values('Período').isin([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])])

                 Aquisição  Comprar  Armazém Fornecedor  Transporte  \
Produto Período                                                       
B1      0              0.0      0.0               149.0         0.0   
        1              0.0      0.0                 0.0         0.0   
        2              0.0      0.0                 0.0         0.0   
        3              0.0      0.0                 0.0         0.0   
        4              0.0      0.0                 0.0         0.0   
        5              0.0      0.0                 0.0         0.0   
        6              0.0      0.0                 0.0         0.0   
        7              0.0      0.0                 0.0         0.0   
        8              0.0      0.0                 0.0         0.0   
        9              0.0      0.0                 0.0         0.0   
B2      0              0.0      0.0               196.0         0.0   
        1              0.0      0.0                 0.0         0.0   
      